In [ ]:
api_key = "YOUR_API_KEY"

In [11]:
from langchain_community.retrievers import PineconeHybridSearchRetriever 

In [16]:
# import os 
# from pinecone import Pinecone , ServerlessSpec
# index_name = "Hybrid-search-langchian-pinecone"

# ## initialize pinecone client
# pc= Pinecone(api_key=api_key)


# index_name = "developer-quickstart-py"

# if not pc.has_index(index_name):
#     pc.create_index_for_model(
#         name=index_name,
        
#         cloud="aws",
#         region="us-east-1",
#         # sparsevalue supported only for dotproduct
#         embed={
#             "model":"llama-text-embed-v2",
#             "field_map":{"text": "chunk_text"},
#             "dimension":384,# embbeding dim 
            
#         }
#     )


In [20]:
import os
from pinecone import Pinecone,ServerlessSpec
index_name="hybrid-search-langchain"
## initialize the Pinecone client
pc=Pinecone(api_key=api_key)

#create the index
if index_name not in pc.list_indexes().names():
    pc.create_index(
        name=index_name,
        dimension=384,  # dimensionality of dense model
        metric="dotproduct",  # sparse values supported only for dotproduct
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )

In [21]:
index=pc.Index(index_name)
index

In [22]:
## vector embedding and sparse matrix
import os
from dotenv import load_dotenv
load_dotenv()

os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.
0it [00:00, ?it/s]
2025-06-03 15:26:42.011342: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748944602.117314  304119 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748944602.147461  304119 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748944602.403705  304119 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False)

In [23]:
# fro sparse matrix convertion 
from pinecone_text.sparse import BM25Encoder

bm25_encoder=BM25Encoder().default()
bm25_encoder

In [24]:
sentences=[
    "In 2023, I visited Paris",
        "In 2022, I visited New York",
        "In 2021, I visited New Orleans",

]

## tfidf values on these sentence
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_values.json")

# load to your BM25Encoder object
bm25_encoder = BM25Encoder().load("bm25_values.json")


100%|██████████| 3/3 [00:00<00:00, 107.19it/s]


In [25]:
retriever=PineconeHybridSearchRetriever(embeddings=embeddings,sparse_encoder=bm25_encoder,index=index)
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, query_encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x7d44a1dd2fe0>, index=<pinecone.db_data.index.Index object at 0x7d46411211e0>)

In [ ]:
retriever.add_texts(
    [
    "In 2023, I visited Paris",
    "In 2022, I visited New York",
    "In 2021, I visited New Orleans",
]
)

100%|██████████| 1/1 [00:02<00:00,  2.20s/it]


In [27]:
retriever.invoke("What city did i visit first")

[Document(metadata={'score': 0.239368573}, page_content='In 2021, I visited New Orleans'),
 Document(metadata={'score': 0.232817784}, page_content='In 2022, I visited New York'),
 Document(metadata={'score': 0.212499827}, page_content='In 2023, I visited Paris')]